In [35]:
from collections import Counter, defaultdict
import math
from PIL import Image
import heapq

In [26]:
#image
from PIL import Image
import heapq
from collections import Counter, defaultdict
import math

class Node:
    def __init__(self, value, freq):
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(frequencies):
    priority_queue = [Node(value, freq) for value, freq in frequencies.items()]
    heapq.heapify(priority_queue)

    while len(priority_queue) > 1:
        left = heapq.heappop(priority_queue)
        right = heapq.heappop(priority_queue)
        merged = Node(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        heapq.heappush(priority_queue, merged)

    return priority_queue[0]

def build_huffman_mapping(node, code='', mapping=defaultdict(str)):
    if node.value is not None:
        mapping[node.value] = code
    else:
        build_huffman_mapping(node.left, code + '0', mapping)
        build_huffman_mapping(node.right, code + '1', mapping)
    return mapping

def huffman_encode(data, mapping):
    encoded_data = ''.join(mapping[value] for value in data)
    return encoded_data

def calculate_bits(message):
    return len(message) * 8  # Assuming 8 bits per pixel

def calculate_compression_ratio(before_bits, after_bits):
    return before_bits / after_bits*100

def calculate_entropy(frequencies):
    total_pixels = sum(frequencies.values())
    entropy = 0
    for freq in frequencies.values():
        probability = freq / total_pixels
        entropy += probability * math.log2(probability)
    return -entropy

def calculate_average_length(mapping):
    total_pixels = sum(len(codeword) for codeword in mapping.values())
    avg_length = total_pixels / len(mapping)
    return avg_length

# Load the image
image_path = input("Enter the path of the image: ").strip('"')  # Strip quotes from the path
image = Image.open(image_path).convert("L")


# Flatten pixel values into a 1D array
pixel_values = list(image.getdata())

# Calculate pixel frequencies
frequencies = Counter(pixel_values)

# Build Huffman tree and mapping
tree = build_huffman_tree(frequencies)
mapping = build_huffman_mapping(tree)

# Encode the image using Huffman coding
encoded_data = huffman_encode(pixel_values, mapping)

# Calculate metrics
original_bits = len(pixel_values) * 8  # Assuming 8 bits per pixel
encoded_bits = len(encoded_data)
compression_ratio = calculate_compression_ratio(original_bits, encoded_bits)
entropy = calculate_entropy(frequencies)
avg_length = calculate_average_length(mapping)
efficiency_of_message= (entropy/avg_length) *100

print("original bits", original_bits)
print("encoded bits bits", encoded_bits)
print("Compression ratio(%):", compression_ratio,"%")
print("Entropy:", entropy ,"bit/bymbol")
print("Average length:", avg_length , "bit/bymbol")
print("efficiency_of_message:" , efficiency_of_message,"%")

Enter the path of the image: "D:\جداريات\FB_IMG_1653879222052.jpg"
original bits 1438208
encoded bits bits 1365474
Compression ratio(%): 105.32664847518151 %
Entropy: 7.565728672777365 bit/bymbol
Average length: 8.979674796747968 bit/bymbol
efficiency_of_message: 84.25392727493127 %


In [50]:

#shahd(Huffman)

import heapq
from collections import Counter, defaultdict
from math import log2

class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(message):
    frequency = Counter(message)
    priority_queue = [Node(char, freq) for char, freq in frequency.items()]
    heapq.heapify(priority_queue)

    while len(priority_queue) > 1:
        left = heapq.heappop(priority_queue)
        right = heapq.heappop(priority_queue)
        merged = Node(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        heapq.heappush(priority_queue, merged)

    return priority_queue[0]

def build_huffman_mapping(node, code='', mapping=defaultdict(str)):
    if node.char is not None:
        mapping[node.char] = code
    else:
        build_huffman_mapping(node.left, code + '0', mapping)
        build_huffman_mapping(node.right, code + '1', mapping)
    return mapping

def huffman_encode(message, mapping):
    encoded_message = ''.join(mapping[char] for char in message)
    return encoded_message

def calculate_bits(message):
    return len(message) * 8  # Assuming 8 bits per character

def calculate_compression_ratio(before_bits, after_bits):
    return before_bits / after_bits



def calculate_entropy(frequencies, total_symbols):
    entropy = 0.0
    for freq in frequencies.values():
        probability = freq / total_symbols
        entropy -= probability * (probability and log2(probability))
    return entropy

def calculate_average_length(mapping, frequencies):
    total_length = sum(len(code) * freq for char, freq in frequencies.items() for code in mapping[char])
    return total_length

def calculate_efficiency(entropy, average_length):
    return entropy / average_length*100

# Example usage
message = input("Enter your message here: ")
tree = build_huffman_tree(message)
mapping = build_huffman_mapping(tree)
encoded_message = huffman_encode(message, mapping)

frequencies = Counter(message)
total_symbols = sum(frequencies.values())

before_bits = calculate_bits(message)
after_bits = len(encoded_message)

compression_ratio = calculate_compression_ratio(before_bits, after_bits)
entropy = calculate_entropy(frequencies, total_symbols)
average_length = calculate_average_length(mapping, frequencies)
efficiency = calculate_efficiency(entropy, average_length)

print("Original Message:", message)
print("Encoded Message:", encoded_message)
print("Total bits before encoding:", before_bits)
print("Total bits after encoding:", after_bits)
print("Compression ratio(%):", compression_ratio)
print("Entropy:", entropy)
print("Average Length:", average_length)
print("Efficiency:", efficiency,"%")


Enter your message here: abcd
Original Message: abcd
Encoded Message: 01110010
Total bits before encoding: 32
Total bits after encoding: 8
Compression ratio(%): 4.0
Entropy: 2.0
Average Length: 8
Efficiency: 25.0 %


In [38]:
# Golomb-Rice encoding and decoding

def golomb_rice_encode(value, M):
   
    quotient = value // M
    remainder = value % M
    
    # Encode quotient in unary; a sequence of 1s followed by a 0.
    unary = '1' * quotient + '0'
    
    # Encode remainder in binary. The length of the binary representation is log2(M).
    binary_length = M.bit_length() - 1
    binary = format(remainder, f'0{binary_length}b')
    
    return unary + binary

def golomb_rice_decode(encoded, M):
   
    # Split the encoded string into unary (quotient) and binary (remainder) parts.
    quotient = 0
    while encoded[quotient] == '1':
        quotient += 1
    encoded = encoded[quotient + 1:]  # Skip over the unary part and the separator '0'.
    
    binary_length = M.bit_length() - 1
    remainder = int(encoded[:binary_length], 2)
    
    value = quotient * M + remainder
    return value


def calculate_bits_before_compression(value):
     if value == 0:
        return 1  # Special case: zero requires one bit ('0')
     else:
            
        return value.bit_length()


def calculate_bits_after_compression(encoded_data):
    bits_after = len(encoded_data)
    return bits_after

def calculate_compression_ratio(bits_before, bits_after):
    compression_ratio = bits_before / bits_after
    return compression_ratio

def calculate_average_length(encoded_data, probabilities):
    average_length = sum(len(encoded_data[data_value]) * probabilities[data_value] for data_value in encoded_data)
    return average_length

def calculate_probabilities(data):
    total_count = len(data)
    probabilities = {value: data.count(value) / total_count for value in set(data)}
    return probabilities

def calculate_entropy(data):
    
    data_length = len(data)
    counter = Counter(data)
    probabilities = [counter[value] / data_length for value in counter]
    entropy = -sum(p * math.log2(p) for p in probabilities if p > 0)
    return entropy

def calculate_efficiency(average_length, entropy):
    efficiency = entropy / average_length * 100
    return efficiency
# Golomb-Rice Example usage:

# Input validation for m and value (must be an integer)
while True:
    try:
        M = int(input("Enter the M parameter (must be an integer) : "))
        break  # Valid input, break the loop
    except ValueError:
        print("Invalid input. M must be an integer  Please try again.")

while True:
    try:
        value = int(input("Enter the value you want to encode (must be an integer): "))
        break  # Valid input, break the loop
    except ValueError:
        print("Invalid input. Value must be an integer. Please try again.")
#M = int(input("Enter The M parameter here: ")) 
#value = int(input("Enter the value you want to encode here: "))  # Value to encode and then decode.

encoded = golomb_rice_encode(value, M)
decoded = golomb_rice_decode(encoded, M)

print("Encoding:", encoded)
print("Decoding:",decoded)


data = [value]  # For probability calculations, consider a list of values
probabilities = calculate_probabilities(data)
bits_before = calculate_bits_before_compression(value)
bits_after = calculate_bits_after_compression(encoded)
compression_ratio = calculate_compression_ratio(bits_before, bits_after)
average_length = calculate_average_length({value: encoded}, probabilities)
entropy= calculate_entropy(encoded)
efficiency= calculate_efficiency(average_length, entropy)

print("Bits Before Compression: ", bits_before)
print("Bits After Compression: ", bits_after)
print("Compression Ratio: ", compression_ratio)
print("Average Length: ", average_length)
print("Probability of Occurrence: ", probabilities[value])
print("Entropy for encoding: ", entropy, "bit/bymbol")
print("Efficiency: ", efficiency, "%")

Enter the M parameter (must be an integer) : 4
Enter the value you want to encode (must be an integer): 8
Encoding: 11000
Decoding: 8
Bits Before Compression:  4
Bits After Compression:  5
Compression Ratio:  0.8
Average Length:  5.0
Probability of Occurrence:  1.0
Entropy for encoding:  0.9709505944546686 bit/bymbol
Efficiency:  19.419011889093373 %


In [39]:
#LZW compression technique

def lzw_compress(data):
    dictionary = {chr(i): i for i in range(128)}
    result = []
    w = ""
    
    for c in data:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            dictionary[wc] = len(dictionary)
            w = c
    
    if w:
        result.append(dictionary[w])
    
    return result

def calculate_bits_after_compression(compressed_data):
    # Determine the maximum bit length needed to represent the compressed codes
    max_bit_length = max(compressed_data).bit_length() if compressed_data else 0
    
    # Calculate total bits used by all compressed codes
    bits_after = len(compressed_data) * max_bit_length
    
    return bits_after

def calculate_bits_before_compression(data):
    bits_before = (len(data) * 8)
    return bits_before

def calculate_probability_of_occurrence(message):
    
    data_length = len(message)
    counter = Counter(message)
    probabilities = {code: counter[code] / data_length for code in counter}
    return probabilities

def calculate_average_code_length(data):
    
    data_length = len(data)
    counter = Counter(data)
    total_bits = sum(len(bin(code)) - 2 for code in set(data))  # Calculate total bits used by all codes
    average_length = total_bits / data_length
    return average_length

def calculate_entropy(data):
 
    data_length = len(data)
    counter = Counter(data)
    probabilities = [counter[code] / data_length for code in counter]
    entropy = -sum(p * math.log2(p) for p in probabilities if p > 0)
    return entropy

def calculate_efficiency(average_length, entropy):
    efficiency = entropy / average_length * 100
    return efficiency
# LZW Example usage:

while True:
    message = input("Enter your message here: ")
    
    # Check if input message is a valid string (not an integer)
    if message.isdigit():
        print("Invalid input. Please enter a valid string message.")
    else:
        break  # Proceed if input is a valid string
        
compressed_data = lzw_compress(message)
bits_before = calculate_bits_before_compression(message)
bits_after = calculate_bits_after_compression(compressed_data)
probabilities = calculate_probability_of_occurrence(message)
entropy = calculate_entropy(message)
average_length = calculate_average_code_length(compressed_data)
efficiency= calculate_efficiency(average_length, entropy)


print("Compressed message: ", compressed_data)
print("Bits before compression: ", bits_before)
print("Bits after compression: ", bits_after)
print("Compression ratio: ", round((bits_before / bits_after) * 100, 2), "%")
print("Probability of occurrence: ", probabilities)
print("Average length: ", average_length )
print("Entropy: ", entropy, "bit/bymbol" )
print("Efficiency: ", efficiency, "%")

Enter your message here: abcd
Compressed message:  [97, 98, 99, 100]
Bits before compression:  32
Bits after compression:  28
Compression ratio:  114.29 %
Probability of occurrence:  {'a': 0.25, 'b': 0.25, 'c': 0.25, 'd': 0.25}
Average length:  7.0
Entropy:  2.0 bit/bymbol
Efficiency:  28.57142857142857 %


In [42]:
#RLE
from collections import Counter
import math


def run_length_encode(message):
    encoded_message = []
    i = 0
    while i < len(message):
        char = message[i]
        count = 1
        while (i + 1 < len(message)) and (message[i + 1] == char):
            count += 1
            i += 1
        encoded_message.append((char, count))
        i += 1
    return encoded_message

def is_numeric_message(encoded_message):
    return all(char.isdigit() for char, _ in encoded_message)

def calculate_bits_before(message):
    if isinstance(message, str):
        if all(char.isdigit() for char in message):
            return len(message)  # Each digit represents 1 bit
        else:
            return 8 * len(message)  # Each character represents 8 bits (ASCII)
    else:
        return message

def calculate_bits_after(encoded_message):
    # Calculate total bits after encoding using RLE
    if is_numeric_message(encoded_message):
        num_segments = len(encoded_message)
        max_count = max(count for _, count in encoded_message) if encoded_message else 0
        bits = math.ceil(math.log2(max_count + 1))
        return num_segments * (1 + bits)
    else:
        bits = 0
        max_count = max(count for _, count in encoded_message)
        max_count_bits = math.ceil(math.log2(max_count + 1))  # Bits needed to represent max count
        for char, count in encoded_message:
            bits += 8  # Bits for representing character (assuming 8-bit ASCII)
            bits += max_count_bits  # Bits for representing the count
        return bits


def calculate_compression_ratio(bits_before, bits_after):
    if bits_before == 0:
        return 0
    return (bits_before  / bits_after)*100

def calculate_probabilities(message):
    char_counts = Counter(message)
    total_chars = len(message)
    probabilities = {char: count / total_chars for char, count in char_counts.items()}
    return probabilities

def calculate_entropy(probabilities):
    entropy = -sum(prob * math.log2(prob) for prob in probabilities.values() if prob > 0)
    return entropy

def calculate_average_length(encoded_message, message_length):
    total_encoded_bits = calculate_bits_after(encoded_message)
    return total_encoded_bits / message_length

def calculate_efficiency(entropy, average_length):
    if entropy == 0:
        return 0
    return entropy / average_length*100

def main():
    message = input("Enter your message: ")

    encoded_message = run_length_encode(message)
    message_length = len(message)

    bits_before = calculate_bits_before(message)
    bits_after = calculate_bits_after(encoded_message)
    compression_ratio = calculate_compression_ratio(bits_before, bits_after)

    probabilities = calculate_probabilities(message)
    entropy = calculate_entropy(probabilities)
    average_length = calculate_average_length(encoded_message, message_length)
    efficiency = calculate_efficiency(entropy, average_length)

    print("Bits before encoding:", bits_before)
    print("Bits after encoding:", bits_after)
    print("Compression ratio(%) :", compression_ratio,"%")
    print("Probability of occurrence for each character:")
    for char, prob in probabilities.items():
        print(f"'{char}': {prob:.4f}")
    print("Entropy:", entropy)
    print("Average length of encoded message:", average_length)
    print("Efficiency:", efficiency,"%")
    print("Encoded message:", encoded_message)

if __name__ == "__main__":
    main()


Enter your message: aaaabbbccccccddd
Bits before encoding: 128
Bits after encoding: 44
Compression ratio(%) : 290.90909090909093 %
Probability of occurrence for each character:
'a': 0.2500
'b': 0.1875
'c': 0.3750
'd': 0.1875
Entropy: 1.936278124459133
Average length of encoded message: 2.75
Efficiency: 70.41011361669575 %
Encoded message: [('a', 4), ('b', 3), ('c', 6), ('d', 3)]


In [51]:
import math

def arithmetic_encode(message, probabilities):
    low = 0.0
    high = 1.0
    range_size = 1.0

    for char in message:
        prob = probabilities[char]
        low += range_size * low_bound(prob)
        high = low + range_size * prob
        range_size = high - low

    return low

def low_bound(probability):
    return probability

def high_bound(probability):
    return probability

def calculate_bits_before(message):
    return len(message) * 8  # Assuming 8 bits per character

def calculate_bits_after(encoded_message):
    # Convert the float number to its binary representation and calculate the length
    binary_representation = bin(int(encoded_message * (2**32)))[2:]  # Convert float to binary string
    return len(binary_representation)

def calculate_compression_ratio(bits_before, bits_after):
    return bits_before / bits_after

def calculate_probabilities(message):
    char_counts = Counter(message)
    total_chars = len(message)
    probabilities = {char: count / total_chars for char, count in char_counts.items()}
    return probabilities

def calculate_entropy(probabilities):
    entropy = -sum(prob * math.log2(prob) for prob in probabilities.values() if prob > 0)
    return entropy

def calculate_average_length(encoded_message, message_length):
    total_encoded_bits = calculate_bits_after(encoded_message)
    return total_encoded_bits / message_length

def calculate_efficiency(entropy, average_length):
    return entropy / average_length*100





def main():
    message = input("Enter your message: ")

    probabilities = calculate_probabilities(message)
    encoded_message = arithmetic_encode(message, probabilities)

    bits_before = calculate_bits_before(message)
    bits_after = calculate_bits_after(encoded_message)
    compression_ratio = calculate_compression_ratio(bits_before, bits_after)

    entropy = calculate_entropy(probabilities)
    average_length = calculate_average_length(encoded_message, len(message))
    efficiency = calculate_efficiency(average_length, entropy)

    print("Bits before encoding:", bits_before)
    print("Bits after encoding:", bits_after)
    print("Compression ratio (%):", compression_ratio)
    print("Entropy:", entropy)
    print("Average length of encoded message:", average_length)
    print("Efficiency(%):", efficiency,"%")
    print("Encoded message:", encoded_message)

if __name__ == "__main__":
    main()


Enter your message: hello
Bits before encoding: 40
Bits after encoding: 31
Compression ratio (%): 1.2903225806451613
Entropy: 1.9219280948873623
Average length of encoded message: 6.2
Efficiency(%): 322.59271387379147 %
Encoded message: 0.26368


In [67]:
############داااااااااااااااااااااااا الصحححححححححححححح
from PIL import Image
import tkinter as tk
from tkinter import messagebox, filedialog
from collections import Counter, defaultdict
import heapq
import math

class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(message):
    frequency = Counter(message)
    priority_queue = [Node(char, freq) for char, freq in frequency.items()]
    heapq.heapify(priority_queue)

    while len(priority_queue) > 1:
        left = heapq.heappop(priority_queue)
        right = heapq.heappop(priority_queue)
        merged = Node(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        heapq.heappush(priority_queue, merged)

    return priority_queue[0]

def build_huffman_mapping(node, code='', mapping=defaultdict(str)):
    if node.char is not None:
        mapping[node.char] = code
    else:
        build_huffman_mapping(node.left, code + '0', mapping)
        build_huffman_mapping(node.right, code + '1', mapping)
    return mapping

def huffman_encode(message, mapping):
    encoded_message = ''.join(mapping[char] for char in message)
    return encoded_message

def calculate_bits(message):
    return len(message) * 8  # Assuming 8 bits per character

def calculate_compression_ratio(before_bits, after_bits):
    return before_bits / after_bits

def calculate_entropy(frequency):
    total_chars = sum(frequency.values())
    entropy = 0
    for count in frequency.values():
        probability = count / total_chars
        entropy += probability * math.log2(probability)
    return -entropy

def calculate_average_length(mapping, frequencies):
    total_length = sum(len(code) * freq for char, freq in frequencies.items() for code in mapping[char])
    return total_length


# LZW compression functions
def lzw_compress(data):
    dictionary = {chr(i): i for i in range(128)}
    result = []
    w = ""

    for c in data:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            dictionary[wc] = len(dictionary)
            w = c

    if w:
        result.append(dictionary[w])

    return result

def calculate_bits_before_compression(message):
    # Assuming 8 bits per character
    return len(message) * 8

def calculate_bits_after_compression(compressed_data):
    # Assuming each code in compressed_data uses 8 bits (modify based on actual implementation)
    return len(compressed_data) * 8

def calculate_average_code_length(compressed_data):
    total_bits = len(compressed_data) * 8
    total_symbols = len(compressed_data)
    average_length = total_bits / total_symbols
    return average_length

def calculate_efficiency(average_length, entropy):
    efficiency = (-entropy / average_length) * 100
    return efficiency



def calculate_probabilities(message):
    frequency = Counter(message)
    total_chars = sum(frequency.values())
    probabilities = {char: freq / total_chars for char, freq in frequency.items()}
    return probabilities

def arithmetic_encode(message, probabilities):
    low = 0.0
    high = 1.0
    range_size = 1.0

    for char in message:
        prob = probabilities[char]
        low += range_size * low_bound(prob)
        high = low + range_size * prob
        range_size = high - low

    return low

def low_bound(probability):
    return probability

def high_bound(probability):
    return probability


def calculate_bits_after_arithmetic_encoding(encoded_message, probabilities):
    # Convert the float number to its binary representation and calculate the length
    binary_representation = bin(int(encoded_message * (2**32)))[2:]  # Convert float to binary string
    return len(binary_representation)

def run_length_encode(message):
    encoded_message = []
    i = 0
    while i < len(message):
        char = message[i]
        count = 1
        while (i + 1 < len(message)) and (message[i + 1] == char):
            count += 1
            i += 1
        encoded_message.append((char, count))
        i += 1
    return encoded_message

def is_numeric_message(encoded_message):
    return all(char.isdigit() for char, _ in encoded_message)

def calculate_bits_before_rle(message):
    if isinstance(message, str):
        if all(char.isdigit() for char in message):
            return len(message)  # Each digit represents 1 bit
        else:
            return 8 * len(message)  # Each character represents 8 bits (ASCII)
    else:
        return message

def calculate_bits_after_rle(encoded_message):
    if is_numeric_message(encoded_message):
        num_segments = len(encoded_message)
        max_count = max(count for _, count in encoded_message) if encoded_message else 0
        bits = math.ceil(math.log2(max_count + 1))
        return num_segments * (1 + bits)
    else:
        bits = 0
        for char, count in encoded_message:
            char_bits = len(format(ord(char), 'b'))  # Bits needed to represent the character
            count_bits = math.ceil(math.log2(count + 1))  # Bits needed to represent the count
            bits += char_bits  # Bits for representing the character
            bits += count_bits  # Bits for representing the count
        return bits
    
def calculate_compression_ratio(bits_before, bits_after):
    if bits_before == 0:
        return 0
    return (bits_before  / bits_after)*100

def calculate_probabilities(message):
    char_counts = Counter(message)
    total_chars = len(message)
    probabilities = {char: count / total_chars for char, count in char_counts.items()}
    return probabilities





class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.pack()
        self.create_widgets()

    def create_widgets(self):
        self.input_type_var = tk.StringVar(value="Text")
        
        self.text_radio_button = tk.Radiobutton(self, text="Text", variable=self.input_type_var, value="Text")
        self.text_radio_button.pack(pady=5)
        
        self.integer_radio_button = tk.Radiobutton(self, text="Integer", variable=self.input_type_var, value="Integer")
        self.integer_radio_button.pack(pady=5)
        
        self.image_radio_button = tk.Radiobutton(self, text="Image", variable=self.input_type_var, value="Image")
        self.image_radio_button.pack(pady=5)
        
        self.input_entry = tk.Entry(self, width=50)
        self.input_entry.pack(pady=10)
        
        self.m_entry_label = tk.Label(self, text="Enter value for m (Golomb-Rice):")
        self.m_entry_label.pack(pady=5)

        self.m_entry = tk.Entry(self, width=10)
        self.m_entry.pack(pady=5)

        self.n_entry_label = tk.Label(self, text="Enter value for n (Golomb-Rice):")
        self.n_entry_label.pack(pady=5)

        self.n_entry = tk.Entry(self, width=10)
        self.n_entry.pack(pady=5)

        self.encode_button = tk.Button(self, text="Encode", command=self.encode_message)
        self.encode_button.pack(pady=5)

        self.output_text = tk.Text(self, width=80, height=20)
        self.output_text.pack()

    def encode_message(self):
        input_type = self.input_type_var.get()
        input_data = self.input_entry.get()

        if not input_data:
            messagebox.showerror("Error", "Please enter some data to encode.")
            return

        try:
            if input_type == "Text":
                self.display_results(input_data)
            elif input_type == "Integer":
                m = self.m_entry.get()
                n = self.n_entry.get()
                if m.isdigit() and n.isdigit():
                    m = int(m)
                    n = int(n)
                    self.display_golomb_rice_results(input_data, m, n)
                else:
                    messagebox.showerror("Error", "Please enter integer values for m and n.")
            elif input_type == "Image":
                self.select_image()
            else:
                messagebox.showerror("Error", "Invalid input type selection.")
        except Exception as e:
            messagebox.showerror("Error", f"Encoding failed: {str(e)}")

    def select_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            self.display_huffman_results(file_path)

    def display_results(self, input_data):
        self.output_text.delete(1.0, tk.END)
        self.output_text.insert(tk.END, f"**Compression Results for Text Input: {input_data}**\n\n")
        # RLE Encoding
        self.rle_encoded_message, self.rle_metrics = self.rle_encode(input_data)
        self.display_result("RLE", self.rle_encoded_message, self.rle_metrics)

        # Huffman Encoding
        self.huffman_encoded_message, self.huffman_metrics = self.huffman_encode(input_data)
        self.display_result("Huffman", self.huffman_encoded_message, self.huffman_metrics)

        # Arithmetic Encoding
        self.arithmetic_encoded_message, self.arithmetic_metrics = self.arithmetic_encode(input_data)
        self.display_result("Arithmetic", self.arithmetic_encoded_message, self.arithmetic_metrics)

        # LZW Encoding
        self.lzw_encoded_message, self.lzw_metrics = self.lzw_encode(input_data)
        self.display_result("LZW", self.lzw_encoded_message, self.lzw_metrics)

        # Determine the optimal technique
        optimal = min(self.huffman_metrics['bits_after'], self.rle_metrics['bits_after'], self.lzw_metrics['bits_after'], self.arithmetic_metrics['bits_after'])
        print("the optimal technique for your message is: ")
        if optimal == self.huffman_metrics['bits_after']:
            print("Huffman coding")
        if optimal == self.arithmetic_metrics['bits_after']:
            print("Arithmetic coding")
        if optimal == self.lzw_metrics['bits_after']:
            print("LZW compression")
        if optimal == self.rle_metrics['bits_after']:
            print("Run-length encoding")


    def display_golomb_rice_results(self, input_data, m, n):
        self.output_text.delete(1.0, tk.END)
        self.output_text.insert(tk.END, f"**Compression Results for Integer Input: {input_data}**\n\n")
        # RLE Encoding
        rle_encoded_message, rle_metrics = self.rle_encode(input_data)
        self.display_result("RLE", rle_encoded_message, rle_metrics)

        # Huffman Encoding
        huffman_encoded_message, huffman_metrics = self.huffman_encode(input_data)
        self.display_result("Huffman", huffman_encoded_message, huffman_metrics)

        # Arithmetic Encoding
        arithmetic_encoded_message, arithmetic_metrics = self.arithmetic_encode(input_data)
        self.display_result("Arithmetic", arithmetic_encoded_message, arithmetic_metrics)

        # Golomb-Rice Encoding
        golomb_rice_encoded_message, golomb_rice_metrics = self.golomb_rice_encode(input_data, m, n)
        self.display_result("Golomb-Rice", golomb_rice_encoded_message, golomb_rice_metrics)

    def display_huffman_results(self, image_path):
        self.output_text.delete(1.0, tk.END)
        self.output_text.insert(tk.END, f"**Compression Results for Image Input: {image_path}**\n\n")
        compress_image(image_path)

    def display_result(self, technique, encoded_message, metrics):
        self.output_text.insert(tk.END, f"**{technique} Encoding Results**\n\n")
        self.output_text.insert(tk.END, f"Encoded Message: {metrics['encoded_message']}\n")
        self.output_text.insert(tk.END, f"Total bits before encoding: {metrics['bits_before']}\n")
        self.output_text.insert(tk.END, f"Total bits after encoding: {metrics['bits_after']}\n")
        self.output_text.insert(tk.END, f"Compression ratio: {metrics['compression_ratio']}\n")
        self.output_text.insert(tk.END, f"Entropy: {metrics['entropy']} bit/symbol\n")
        if 'average_length' in metrics:
            self.output_text.insert(tk.END, f"Average length of encoded message: {metrics['average_length']} bit/symbol\n")
            self.output_text.insert(tk.END, f"Efficiency: {metrics['efficiency_of_message']}%\n\n")

    def rle_encode(self, input_data):
        # Implement RLE encoding here    
        encoded_message = run_length_encode(input_data) # Placeholder
        bits_before = calculate_bits_before_rle(input_data)
        bits_after = calculate_bits_after_rle(encoded_message)
        compression_ratio = calculate_compression_ratio(bits_before, bits_after)
        probabilities = calculate_probabilities(input_data)
        entropy = calculate_entropy(probabilities)
        average_length = average_length= bits_after / len(input_data)
        efficiency_of_message = (entropy / average_length) * 100
    
    
    
        return encoded_message, {
            "encoded_message": encoded_message,
            "bits_before": bits_before,
            "bits_after": bits_after,
            "compression_ratio": compression_ratio,
            "entropy": entropy,
            "average_length": average_length,
            "efficiency_of_message": efficiency_of_message
        }
    

    def huffman_encode(self, input_data):
            tree = build_huffman_tree(input_data)
            mapping = build_huffman_mapping(tree)
            encoded_message = huffman_encode(input_data, mapping)
            frequencies = Counter(input_data)
            bits_before = calculate_bits(input_data)
            bits_after = len(encoded_message)
            compression_ratio = calculate_compression_ratio(bits_before, bits_after)
            entropy = calculate_entropy(Counter(input_data))
            average_length = calculate_average_length(mapping, frequencies)
            efficiency_of_message = calculate_efficiency(entropy, average_length)

            
            return encoded_message, {
                "encoded_message": encoded_message,
                "bits_before": bits_before,
                "bits_after": bits_after,
                "compression_ratio": compression_ratio,
                "entropy": entropy,
                "average_length": average_length,
                "efficiency_of_message":  entropy / average_length*100
            }


    def arithmetic_encode(self, input_data):
        probabilities = calculate_probabilities(input_data)
        encoded_message = arithmetic_encode(input_data, probabilities)
        bits_before = calculate_bits(input_data)
        bits_after = calculate_bits_after_arithmetic_encoding(encoded_message, probabilities)
        compression_ratio = calculate_compression_ratio(bits_before, bits_after)
        entropy = calculate_entropy(probabilities)
        average_length=bits_after /len(input_data)
        efficiency_of_message = (entropy / average_length) * 100
        return encoded_message, {
            "encoded_message": encoded_message,
            "bits_before": bits_before,
            "bits_after": bits_after,
            "compression_ratio": compression_ratio,
            "average_length":average_length,
            "entropy": entropy,
            "efficiency_of_message":entropy / average_length*100
        }

    def lzw_encode(self, input_data):
        compressed_data = lzw_compress(input_data)
        frequency = Counter(compressed_data)
        bits_before = calculate_bits_before_compression(input_data)
        bits_after = calculate_bits_after_compression(compressed_data)
        compression_ratio = calculate_compression_ratio(bits_before, bits_after)
        entropy = calculate_entropy(frequency)
        average_length = calculate_average_code_length(compressed_data)
        efficiency_of_message = (entropy / average_length) * 100
        return compressed_data, {
            "encoded_message": compressed_data,
            "bits_before": bits_before,
            "bits_after": bits_after,
            "compression_ratio": compression_ratio,
            "entropy": entropy,
            "average_length": average_length,
            "efficiency_of_message": efficiency_of_message
        }

    def golomb_rice_encode(self, input_data, m, n):
        if not isinstance(m, int) or not isinstance(n, int):
            raise ValueError("Parameters m and n must be integers.")
    
        # Perform Golomb-Rice encoding using m and n
        encoded_message = input_data  # Placeholder
        bits_before = calculate_bits(input_data)
        bits_after = calculate_bits(encoded_message)
        compression_ratio = calculate_compression_ratio(bits_before, bits_after)
        entropy = calculate_entropy(Counter(input_data))
        average_length = len(input_data)
        efficiency_of_message = (entropy / average_length) * 100
        return encoded_message, {
            "encoded_message": encoded_message,
            "bits_before": bits_before,
            "bits_after": bits_after,
            "compression_ratio": compression_ratio,
            "entropy": entropy,
            "average_length": average_length,
            "efficiency_of_message": efficiency_of_message
        }
    
    
    
def compress_image(image_path):
    
    # Load the image
    image = Image.open(image_path).convert("L")

    # Flatten pixel values into a 1D array
    pixel_values = list(image.getdata())

    # Calculate pixel frequencies
    frequencies = Counter(pixel_values)

    # Build Huffman tree and mapping
    tree = build_huffman_tree(frequencies)
    mapping = build_huffman_mapping(tree)

    # Encode the image using Huffman coding
    encoded_data = huffman_encode(pixel_values, mapping)

    # Calculate metrics
    original_bits = len(pixel_values) * 8  # Assuming 8 bits per pixel
    encoded_bits = len(encoded_data)
    compression_ratio = calculate_compression_ratio(original_bits, encoded_bits)
    total_pixels = sum(len(codeword) for codeword in mapping.values())
    entropy = calculate_entropy(frequencies)
    avg_length = total_pixels / len(mapping)
    efficiency_of_message = (entropy / avg_length) * 100

    print("Compression ratio:", compression_ratio)
    print("Entropy:", entropy, "bit/bymbol")
    print("Average length:", avg_length, "bit/bymbol")
    print("Efficiency of message:", efficiency_of_message, "%")

root = tk.Tk()
root.title("Compression Results")
app = Application(master=root)
app.mainloop()
